In [2]:
import findspark
findspark.init('/home/sushant/spark-2.1.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('MLspark').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [7]:
!pwd

/home/sushant/Documents/SparkUdemyCourse


In [8]:
training = spark.read.format('libsvm').load('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt')

In [10]:
training.show()
# note that we have a label column and a feature column.

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

The feature column inside of it has a vector and that is a vector of all te features that belong to that row. 

In [11]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')
# These are the default values for featuresCol, labelCol, and predictionCol. We can change them if we want.

In [12]:
lrModel = lr.fit(training)

In [14]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [15]:
lrModel.intercept

0.14228558260358093

In [16]:
trainSummary = lrModel.summary

In [17]:
trainSummary.r2

0.027839179518600154

Not a great model. 

In [18]:
trainSummary.rootMeanSquaredError

10.16309157133015

### Test train split

In [20]:
entire_data = spark.read.format('libsvm').load('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt')

In [24]:
train_data, test_data = entire_data.randomSplit([0.7, 0.3])

In [25]:
test_data.count()

159

In [26]:
train_data.count()

342

In [27]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.08565904102149|(10,[0,1,2,3,4,5,...|
|-15.951512565794573|(10,[0,1,2,3,4,5,...|
|-15.732088272239245|(10,[0,1,2,3,4,5,...|
| -15.72351561304857|(10,[0,1,2,3,4,5,...|
|-15.334767479922341|(10,[0,1,2,3,4,5,...|
|-15.310980589416289|(10,[0,1,2,3,4,5,...|
| -13.15333560636553|(10,[0,1,2,3,4,5,...|
|-12.558575788856189|(10,[0,1,2,3,4,5,...|
|-12.491442077546413|(10,[0,1,2,3,4,5,...|
|-11.857350365429426|(10,[0,1,2,3,4,5,...|
+----------

In [37]:
correct_model = lr.fit(train_data)

In [38]:
test_results = correct_model.evaluate(test_data)

In [39]:
test_results.r2

-0.0026225007202784756

Model will be deployed on data without any labels. 

In [40]:
unlabeled_data = test_data.select('features')

In [41]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [42]:
predictions = correct_model.transform(unlabeled_data)

In [43]:
predictions.show(2)

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-0.4054524588437125|
|(10,[0,1,2,3,4,5,...| 2.0465011230675487|
+--------------------+-------------------+
only showing top 2 rows



In [36]:
test_data.show(2)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 2 rows



### Working with a messy data

In [48]:
data = spark.read.csv('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv', 
                          inferSchema=True, header=True)

In [51]:
data.show(3)

+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|   Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|   Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|   Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
+--------------------+--------------------+---------+------------------+----------------

In [52]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [54]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



The target variable here is the Yearly Amount Spent.

In [55]:
data.head(2)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.92627202636016, Time on App=11.109460728682564, Time on Website=37.268958868297744, Length of Membership=2.66403418213262, Yearly Amount Spent=392.2049334443264)]

In [58]:
data.head(2)[1][2]

'DarkGreen'

Now let's convert this to the input format that Spark needs. 

In [60]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

For now we work only with the numeric data

In [61]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [65]:
assembler = VectorAssembler(inputCols = ['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], 
                           outputCol = 'features')

In [66]:
output = assembler.transform(data)
# note: we are passing all the data

In [67]:
output.show(4)

+--------------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|     Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|     Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|  DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|     Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.10454320

In [68]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [71]:
output.select('features').show(4)

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
+--------------------+
only showing top 4 rows



In [72]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [74]:
final_data = output.select(['features', 'Yearly Amount Spent'])

In [75]:
final_data.show(3)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
+--------------------+-------------------+
only showing top 3 rows



In [76]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [79]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                352|
|   mean|  498.1534381176247|
| stddev|  81.38737902814381|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [80]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                148|
|   mean|  502.0743845398077|
| stddev|  74.34676082249456|
|    min|  282.4712457199145|
|    max|  689.2356997616951|
+-------+-------------------+



Now, train the regression model. 

In [81]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [82]:
lr_model = lr.fit(train_data)

In [83]:
test_results = lr_model.evaluate(test_data)

In [84]:
test_results.rootMeanSquaredError
# compare this number with mean, and stddev of the target variable seen in the summary below. 
# The smallness of rootMeanSquaredError tells that the model is pretty good. 

9.331837562122383

In [85]:
test_results.r2

0.9841381215256287

In [88]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [89]:
unlabeled_data = test_data.select('features')

In [90]:
unlabeled_data.show(4)

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[31.1239743499119...|
|[31.2834474760581...|
|[31.3584771924370...|
+--------------------+
only showing top 4 rows



In [91]:
predictions = lr_model.transform(unlabeled_data)

In [92]:
predictions.show(4)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|288.63976949347807|
|[31.1239743499119...| 508.1460920922427|
|[31.2834474760581...| 568.9677016306939|
|[31.3584771924370...| 490.9632437324274|
+--------------------+------------------+
only showing top 4 rows



## Project

In [93]:
ship_data = spark.read.csv('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/cruise_ship_info.csv', 
                          inferSchema=True, header=True)

In [95]:
ship_data.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



In [96]:
ship_data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



Gotta convert the cruise line which is string to a numeric value which our ML algorithms can use. 

In [99]:
ship_data.groupBy('Cruise_line').count().orderBy('count').show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|           Orient|    1|
|           Disney|    2|
|          Azamara|    2|
|          Crystal|    2|
|           Cunard|    3|
|         Windstar|    3|
|          Oceania|    3|
|         Seabourn|    3|
|        Silversea|    4|
|Regent_Seven_Seas|    5|
|              P&O|    6|
|             Star|    6|
|              MSC|    8|
|        Celebrity|   10|
|            Costa|   11|
|        Norwegian|   13|
| Holland_American|   14|
|         Princess|   17|
|         Carnival|   22|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [100]:
from pyspark.ml.feature import StringIndexer

In [103]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol = 'Cruise_index')

In [104]:
indexed = indexer.fit(ship_data).transform(ship_data)

In [105]:
indexed.show(4)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|        16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|        16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|         1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|         1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+------------+
only showing top 4 rows



In [107]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_index']

In [135]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins', 
                                       'passenger_density', 'Cruise_index'], 
                            outputCol = 'features')

In [136]:
output = assembler.transform(indexed).select('features', 'crew')

In [137]:
output.show(4)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
+--------------------+----+
only showing top 4 rows



In [138]:
train_data, test_data = output.randomSplit([0.7, 0.3])

In [139]:
train_data.count()

112

In [140]:
test_data.count()

46

In [141]:
lr_ship = LinearRegression(labelCol='crew')

In [142]:
lr_ship_model = lr_ship.fit(train_data)

In [143]:
lr_ship_results = lr_ship_model.evaluate(test_data)

In [144]:
lr_ship_results.rootMeanSquaredError

1.2234570976578782

In [145]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               112|
|   mean| 7.486696428571437|
| stddev|3.2740853583926595|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [146]:
lr_ship_results.r2

0.9017527117967735

Pretty good result for linear regression. This makes sense if two variables are correlated. 

In [147]:
from pyspark.sql.functions import corr

In [148]:
ship_data.show(3)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 3 rows



In [150]:
from pyspark.sql.functions import max, min

In [153]:
ship_data.select(corr('passengers','crew')).show()

+----------------------+
|corr(passengers, crew)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [155]:
ship_data.select(corr('cabins','crew')).show()

+------------------+
|corr(cabins, crew)|
+------------------+
|0.9508226063578497|
+------------------+



Strong correlation! The reason why linear regression works so well.